In [ ]:
from eecs598.transformers.src.transformers import BertGenerationTokenizer, BertGenerationDecoder, BertGenerationConfig,  BertGenerationEncoder

In [ ]:
import torch

In [ ]:
### This is an original experiment with 'BERTGeneration', but it seems like this isn't the model we should use

tokenizer = BertGenerationTokenizer.from_pretrained('google/bert_for_seq_generation_L-24_bbc_encoder')
config = BertGenerationConfig.from_pretrained("google/bert_for_seq_generation_L-24_bbc_encoder")
config_enc = BertGenerationConfig.from_pretrained("google/bert_for_seq_generation_L-24_bbc_encoder")
config.is_decoder = True
model = BertGenerationDecoder.from_pretrained('google/bert_for_seq_generation_L-24_bbc_encoder', config=config)
enc_model = BertGenerationEncoder.from_pretrained('google/bert_for_seq_generation_L-24_bbc_encoder', config=config_enc)

In [ ]:
### Further exploration of BERTGeneration
inputs = tokenizer(["Hello, my dog is cute"], return_tensors="pt")
label_outputs = tokenizer(["and stuff and winaokwnr ajdns"], return_tensors="pt")
print(inputs['input_ids'].shape)
print(label_outputs['input_ids'].shape)
outputs = model(inputs['input_ids'], inputs['attention_mask'], labels=label_outputs['input_ids'])
print(outputs[0])

In [ ]:
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0) 

In [ ]:
outputs = model(**input_ids)

In [ ]:
from eecs598.transformers.src.transformers import EncoderDecoderModel, BertTokenizer

In [ ]:
## This is the Seq2Seq model we want to use for our task, more details here:
## https://huggingface.co/transformers/model_doc/encoderdecoder.html

model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased') # initialize Bert2Bert from pre-trained checkpoints


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print(model)

In [ ]:
input_ids = torch.tensor(tokenizer.encode(["What is the", "and what else"], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
label_outputs = torch.tensor(tokenizer.encode(["name of the gym?", "can we do here"], add_special_tokens=True)).unsqueeze(0)
outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, labels=label_outputs)
print(outputs[0])

In [ ]:
generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder.pad_token_id, do_sample=True)
print(tokenizer.decode(generated[0], skip_special_tokens=True))

In [ ]:
from eecs598.transformers.src.transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
## Simple training script with just a couple examples to demo
## Based on https://huggingface.co/transformers/training.html#pytorch

num_epochs = 10
text_batch = ["I love Pixar.", "I don't care for Pixar."]
input_ids = torch.tensor(tokenizer.encode(text_batch, add_special_tokens=True)).unsqueeze(0)  # Batch size 1
label_outputs = torch.tensor(tokenizer.encode(["so much","at all"], add_special_tokens=True)).unsqueeze(0)  # Batch size 1
for i in range(num_epochs):
    print(i)
    optimizer.zero_grad()
    outputs = model(input_ids=input_ids, decoder_input_ids=input_ids, labels=label_outputs)
    loss = outputs[0]
    loss.backward()
    optimizer.step()